In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from patsy import dmatrix, dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [83]:
df = pd.read_csv("tbl_LotInfo.csv")
df.head()

,nLotID,strLotName,nOperationNumber,nUsersGroupID,strOperatorName,strUsersGroupName,nModelID,strModelName,strStartTime,strEndTime,...,nNormalFlushSheetCount,nForceFlushSheetCount,nResult_A_Count,nResult_B_Count,nGrade_B_Swell_Count,nGrade_B_Soot_Count,nGrade_B_Hole_Count,nGrade_Etc_Count,nGrade_Error_Count,nGrade_B_Discolor_Count
0,1,2.02205E+13,1,1,abc,��1,4,Sample Model ROI,2022-05-04 14:38,2022-05-04 14:39,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2.02205E+13,1,1,abc,��1,4,Sample Model ROI,2022-05-04 14:40,2022-05-04 14:45,...,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2.02205E+13,1,1,abc,��1,4,Sample Model ROI,2022-05-04 14:45,2022-05-04 14:46,...,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2.02205E+13,1,1,abc,��1,4,Sample Model ROI,2022-05-04 14:47,2022-05-04 14:50,...,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2.02205E+13,1,1,abc,��1,4,Sample Model ROI,2022-05-04 14:50,2022-05-04 14:53,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
print(len(df.columns)); df.columns

24


Index(['nLotID', 'strLotName', 'nOperationNumber', 'nUsersGroupID',
       'strOperatorName', 'strUsersGroupName', 'nModelID', 'strModelName',
       'strStartTime', 'strEndTime', 'nReferenceSheetID', 'nStartSheetID',
       'nEndSheetID', 'nTotalSheetCount', 'nNormalFlushSheetCount',
       'nForceFlushSheetCount', 'nResult_A_Count', 'nResult_B_Count',
       'nGrade_B_Swell_Count', 'nGrade_B_Soot_Count', 'nGrade_B_Hole_Count',
       'nGrade_Etc_Count', 'nGrade_Error_Count', 'nGrade_B_Discolor_Count'],
      dtype='object')

In [85]:
# nUsersGroupID
jik_1 = 0
jik_2 = 0
jik_3 = 0
for i in df.nUsersGroupID:
    if i == 1:
        jik_1+=1
    elif i == 2:
        jik_2+=1
    elif i == 3:
        jik_3+=1
    else:
        print(i)
print(jik_1, jik_2, jik_3)
        

5
5
228 178 100


In [86]:
df1 = df.drop(['strLotName','strOperatorName', 'nModelID', 'strModelName'], axis=1)
df1.head()

,nLotID,nOperationNumber,nUsersGroupID,strUsersGroupName,strStartTime,strEndTime,nReferenceSheetID,nStartSheetID,nEndSheetID,nTotalSheetCount,nNormalFlushSheetCount,nForceFlushSheetCount,nResult_A_Count,nResult_B_Count,nGrade_B_Swell_Count,nGrade_B_Soot_Count,nGrade_B_Hole_Count,nGrade_Etc_Count,nGrade_Error_Count,nGrade_B_Discolor_Count
0,1,1,1,��1,2022-05-04 14:38,2022-05-04 14:39,0,-1,-1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,1,��1,2022-05-04 14:40,2022-05-04 14:45,0,1,6,6,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,1,��1,2022-05-04 14:45,2022-05-04 14:46,6,7,12,6,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,1,��1,2022-05-04 14:47,2022-05-04 14:50,12,13,14,2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,1,��1,2022-05-04 14:50,2022-05-04 14:53,14,15,16,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
# datetime으로 type 변경
df1.strStartTime = pd.to_datetime(df1.strStartTime)
df1.strEndTime = pd.to_datetime(df1.strEndTime)

# strEndTime이 결측치인 행 제거
df1 = df1.dropna(subset=['strEndTime'])

# 시간차 columns 추가
df1['timediff'] = (df1.strEndTime-df1.strStartTime).dt.seconds // 60

In [88]:
df1['nTotalSheetCount']

0        0
1        6
2        6
3        2
4        2
      ... 
503    736
504    408
505    202
506    672
507    228
Name: nTotalSheetCount, Length: 506, dtype: int64

In [92]:
df1['timepersheet'] = df1['timediff'] / df1['nTotalSheetCount']
df1['timepersheet']

0           inf
1      0.833333
2      0.166667
3      1.500000
4      1.500000
         ...   
503    0.600543
504    0.526961
505    1.282178
506    0.842262
507    0.521930
Name: timepersheet, Length: 506, dtype: float64

In [113]:
for i in range(len(df1['timepersheet'])):
    if df1['timepersheet'][i] > 1.5:
        print(i)

0


KeyError: 5

In [89]:
df1['timepersheet'] = 0
for i in range(len(df)):
    if df1['nTotalSheetCount'][i] != 0:
        df1['timepersheet'][i] = df1['timediff'][i] / df1['nTotalSheetCount'][i]
    else:
        df1['timepersheet'][i] = df1['timediff'][i]

C:\Users\cmson\AppData\Local\Temp\ipykernel_14492\268583190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['timepersheet'][i] = df1['timediff'][i]
C:\Users\cmson\AppData\Local\Temp\ipykernel_14492\268583190.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['timepersheet'][i] = df1['timediff'][i] / df1['nTotalSheetCount'][i]


KeyError: 5

In [ ]:
'''
1. 시간차 / 검사개수 > 임계값 --> 제외

2. 검사 못 한 것, A, B / 총 검사 수 시각화

'''